# [INFER] Siim-cov19 efnb7 infer (study)

thanks to https://www.kaggle.com/xhlulu/siim-covid-19-convert-to-jpg-256px  
train: https://www.kaggle.com/h053473666/siim-covid19-efnb7-train-study

* model
    1. efficientnetb7
    2. efficientnetV2
        추후에 tfhub.weight 관련 이뤄지면서 작동이 잘 안되었었다.
        생각해보니, effnV2 github 에서 받은 가중치 중에 b 시리즈는 100MB 정도, XL 는 1GB 가 넘어갔다.
        Notebook 에서 수정된 새로운 가중치를 다운로드 받아보니,  
        [[Notebook] Train study level](https://www.kaggle.com/sreevishnudamodaran/siim-effnetv2-keras-study-train-tpu-cv0-805/data)  
        사이즈가 하나당 1GB 가 넘었다. 엊그제 여기서 받았을 때는 대략 100MB 가 안되었다.  
        뭔가 잘못되었던 것 같다.  
        또한, offline 에서 Keras layer 를 맞출 떄, 기존 weight 는 parameter 441 로 b1 모델에 매칭이 되었는데, 이것도 확인해보자.
    3. efficientnetV2_XL (새롭게 수정된 notebook 에서 다운받은 weight)   
        실험해보자. 

[[github] google-automl-efficientnetv2](https://github.com/google/automl/tree/master/efficientnetv2)

### efficientnetV2 

efnbV2 의 경우 import 하는 방법이 2가지 이다.
1. 직접 import (github 가져오려면 인터넷 연결 필요)
2. tfhub 사용 (tfhub 인터넷 연결 필요)
-> 캐글에서 사용하려면 인터넷 연결로 다운로드 하거나, 인터넷 연결없이 dataset 구축 하여 가져오는 방법

구조가 달라, inference 할 경우, tfhub.wrapper 를 시켜줘야 한다.    
[관련 노트북 참조](https://www.kaggle.com/sreevishnudamodaran/siim-effnetv2-l-cascadercnn-mmdetection-infer)

In [1]:
# efnb7 : my_trained_model : 0.428 (caused by resized data? or not)
# efnb7 : pretrained_model : 0.438
# efnv2(l) : pretrained_model : 0.441

In [2]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y


######################################################################## | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

######################################################################## | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

######################################################################## | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

######################################################################## | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

######################################################################## | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

######################################################################## | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm

v1 = False
v2 = not v1

tta = False

# .dcm to .png

In [4]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [5]:
df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
if df.shape[0] == 2477:
    fast_sub = True
    fast_df = pd.DataFrame(([['00086460a852_study', 'negative 1 0 0 1 1'], 
                         ['000c9c05fd14_study', 'negative 1 0 0 1 1'], 
                         ['65761e66de9f_image', 'none 1 0 0 1 1'], 
                         ['51759b5579bc_image', 'none 1 0 0 1 1']]), 
                       columns=['id', 'PredictionString'])
else:
    fast_sub = False
fast_sub

True

In [6]:
split = 'test'
save_dir = f'/kaggle/tmp/{split}/'

study_level_size = 768

os.makedirs(save_dir, exist_ok=True)

# study level 용 이미지 생성 폴더
save_dir = f'/kaggle/tmp/{split}/study/'
os.makedirs(save_dir, exist_ok=True)

if fast_sub:
    xray = read_xray('../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=study_level_size)  
    study = '00086460a852' + '_study.png'
    im.save(os.path.join(save_dir, study))
    xray = read_xray('../input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    im = resize(xray, size=study_level_size)  
    study = '000c9c05fd14' + '_study.png'
    im.save(os.path.join(save_dir, study))
else:   
    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size = study_level_size)  
            study = dirname.split('/')[-2] + '_study.png'
            im.save(os.path.join(save_dir, study))

In [7]:
import numpy as np 
import pandas as pd

if fast_sub:
    df = fast_df.copy()
else:
    df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
    
id_laststr_list = []

for i in range(df.shape[0]):
    id_laststr_list.append(df.loc[i,'id'][-1])
df['id_last_str'] = id_laststr_list

study_len = df[df['id_last_str'] == 'y'].shape[0]

# predict

In [8]:
!pip install /kaggle/input/kerasapplications -q
print('install keras applications package')

install keras applications package


In [9]:
!pip install /kaggle/input/efficientnet-keras-source-code -q --no-deps
print('install efficientnet keras package')
# 패키지 데이터셋 변경으로 인해 다시 가져와서 설치하였음

install efficientnet keras package


In [10]:
import os

import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
import tensorflow as tf

def auto_select_accelerator():
    try:
        print('try')
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        print('except')
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    return strategy


def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg', tta=None):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        if tta == None:
            return img
        elif tta == 'flip':
            return tf.image.flip_left_right(img)
        

    def decode_with_labels(path, label):
        return decode(path), label

    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img

    def augment_with_labels(img, label):
        return augment(img), label

    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)

    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)

    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)

    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)

    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)

    return dset

#COMPETITION_NAME = "siim-cov19-test-img512-study-600"
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16

if v1:
    IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600)
    IMS = 7
elif v2:
    IMSIZE = (768,)
    IMS = 0
target_size = (IMSIZE[IMS], IMSIZE[IMS])

print(f"target_size : {target_size}")

#load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
if fast_sub:
    sub_df = fast_df.copy()
else:
    sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
sub_df = sub_df[:study_len]
test_paths = f'/kaggle/tmp/{split}/study/' + sub_df['id'] +'.png'

sub_df['negative'] = 0
sub_df['typical'] = 0
sub_df['indeterminate'] = 0
sub_df['atypical'] = 0

label_cols = sub_df.columns[2:]

print('build datset')
test_decoder = build_decoder(with_labels=False, 
                             target_size=target_size, 
                             ext='png')

dtest = build_dataset(test_paths, 
                      bsize=BATCH_SIZE, 
                      repeat=False, 
                      shuffle=False, 
                      augment=False, 
                      cache=False,
                      decode_fn=test_decoder)

if tta:
    print('build tta datset')
    test_decoder_flip = build_decoder(with_labels=False, 
                                      target_size=target_size, 
                                      ext='png',
                                      tta='flip')

    dtest_flip = build_dataset(test_paths, 
                               bsize=BATCH_SIZE, 
                               repeat=False, 
                               shuffle=False, 
                               augment=False, 
                               cache=False,
                               decode_fn=test_decoder_flip)

try
except
Running on 1 replicas
target_size : (768, 768)
build datset


In [11]:
print('ready')

ready


In [12]:
# efficientnetV2 불러오기
# GCS 에서 학습되었으므로 학습 경로 구조가 다르다. wrapper class 를 만들어 맞춰줘야 한다.
# https://www.kaggle.com/sreevishnudamodaran/siim-effnetv2-l-cascadercnn-mmdetection-infer

v1 = False
v2 = True


import tensorflow as tf
import tensorflow_hub as tfhub

MODEL_ARCH = 'efficientnetv2-l-21k-ft1k'
# Get the Tensorflow Hub model URL
hub_type = 'feature_vector' # ['classification', 'feature_vector']
MODEL_ARCH_PATH = f'/kaggle/input/efficientnetv2-tfhub-weight-files/tfhub_models/{MODEL_ARCH}/{hub_type}'

# Custom wrapper class to load the right pretrained weights explicitly from the local directory
class KerasLayerWrapper(tfhub.KerasLayer):
    def __init__(self, handle, **kwargs):
        handle = tfhub.KerasLayer(tfhub.load(MODEL_ARCH_PATH))
        super().__init__(handle, **kwargs)

if v2:
    MODEL_PATH = '/kaggle/input/siim-study-efnv2-model/study_efnv2_xl_model'
    with tf.device('/device:GPU:0'): # Specifies the device for ops created/executed in this context.
        models = []
        models0 = tf.keras.models.load_model(f"{MODEL_PATH}/model0.h5",
                                             custom_objects = {'KerasLayer':KerasLayerWrapper})
        models1 = tf.keras.models.load_model(f"{MODEL_PATH}/model1.h5",
                                             custom_objects = {'KerasLayer':KerasLayerWrapper})
        models2 = tf.keras.models.load_model(f"{MODEL_PATH}/model2.h5",
                                             custom_objects = {'KerasLayer':KerasLayerWrapper})
        models3 = tf.keras.models.load_model(f"{MODEL_PATH}/model3.h5",
                                             custom_objects = {'KerasLayer':KerasLayerWrapper})
        models4 = tf.keras.models.load_model(f"{MODEL_PATH}/model4.h5",
                                             custom_objects = {'KerasLayer':KerasLayerWrapper})
        models.append(models0)
        models.append(models1)
        models.append(models2)
        models.append(models3)
        models.append(models4)
    
    print(f"try prediction")
    result1 = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)        
    
    if tta:
        print(f"try flip tta")
        result2 = sum([model.predict(dtest_flip, verbose=1) for model in models]) / len(models)        
        result = result1 * 0.5 + result2 * 0.5
        sub_df[label_cols] = result
    else:
        sub_df[label_cols] = result1
    
    
# tf.keras.models.load_model -> custom_objects : 특정 층을 다른 층으로 바꾸겠다. dict mapping    
# models0 = tf.keras.models.load_model(
#     '/kaggle/input/siim-study-efnv2-model/study_efnv2_model/model0.h5')

try prediction
1/1 [==============================] - 4s 4s/step


In [13]:
print('done')

done


In [14]:
# efnb7 : '../input/siim-covid19-efnb7-train-study/model0.h5'
# efnv2 : '../input/siim-study-efnv2-model/study_efnv2_model/model0.h5'

# v1 : efficientnetV1 (B7)

if v1:
    with strategy.scope():

        models = []

        models0 = tf.keras.models.load_model(
            '../input/siim-study-efnv2-model/study_efnv2_model/model0.h5'
        )
        models1 = tf.keras.models.load_model(
            '../input/siim-study-efnv2-model/study_efnv2_model/model1.h5'
        )
        models2 = tf.keras.models.load_model(
            '../input/siim-study-efnv2-model/study_efnv2_model/model2.h5'
        )
        models3 = tf.keras.models.load_model(
            '../input/siim-study-efnv2-model/study_efnv2_model/model3.h5'
        )
        models4 = tf.keras.models.load_model(
            '../input/siim-study-efnv2-model/study_efnv2_model/model4.h5'
        )

        models.append(models0)
        models.append(models1)
        models.append(models2)
        models.append(models3)
        models.append(models4)

    sub_df[label_cols] = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)

In [15]:
sub_df.columns = ['id', 'PredictionString1', 'negative', 'typical', 'indeterminate', 'atypical']
df = pd.merge(df, sub_df, on = 'id', how = 'left')

# study string

In [16]:
for i in range(study_len):
    negative = df.loc[i,'negative']
    typical = df.loc[i,'typical']
    indeterminate = df.loc[i,'indeterminate']
    atypical = df.loc[i,'atypical']
    df.loc[i, 'PredictionString'] = f'negative {negative} 0 0 1 1 typical {typical} 0 0 1 1 indeterminate {indeterminate} 0 0 1 1 atypical {atypical} 0 0 1 1'

In [17]:
df = df[['id', 'PredictionString']]

df.to_csv('submission.csv',index=False)
df

,id,PredictionString
0,00086460a852_study,negative 0.07245215773582458 0 0 1 1 typical 0...
1,000c9c05fd14_study,negative 0.01065667625516653 0 0 1 1 typical 0...
2,65761e66de9f_image,none 1 0 0 1 1
3,51759b5579bc_image,none 1 0 0 1 1
